In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_csv('Real_Estate_Sales_2001-2020_GL.csv')

C:\Users\kazis\AppData\Local\Temp\ipykernel_26936\1766759599.py:1: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Real_Estate_Sales_2001-2020_GL.csv')


In [ ]:
df.info()

In [3]:
df.head(2)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
1,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,NaN,NaN,NaN,NaN


In [4]:
df[df['Date Recorded'].isna()]

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
177917,20280,2002,NaN,Orange,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
179410,0,2002,NaN,Orange,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.drop([177917, 179410])

In [6]:
df[df['Town'] == '***Unknown***']

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
444657,70086,2007,12/18/2007,***Unknown***,18 MATHIEU LANE,66540.0,282450.0,0.235582,Single Family,Single Family,07 - Change in Property,NaN,NaN,NaN


In [7]:
df = df.drop([444657])

In [8]:
df = df.dropna(subset=['Address'])

In [9]:
df['Address'] = df['Address'].str.replace('\d+', '')

C:\Users\kazis\AppData\Local\Temp\ipykernel_26936\2132601316.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Address'] = df['Address'].str.replace('\d+', '')


In [10]:
df['Property Type'].isna().sum()

382399

In [11]:
df['Property Type'].unique()

array(['Commercial', 'Residential', 'Vacant Land', nan, 'Apartments',
       'Industrial', 'Public Utility', 'Condo', 'Two Family',
       'Three Family', 'Single Family', 'Four Family'], dtype=object)

In [12]:
avg_sales_ratio = df.groupby('Property Type')['Sales Ratio'].mean()

In [13]:
print(avg_sales_ratio)

Property Type
Apartments        0.589576
Commercial        4.304097
Condo             0.859682
Four Family       1.421730
Industrial        1.794958
Public Utility    0.955225
Residential       0.703482
Single Family     8.115140
Three Family      1.444156
Two Family        1.213065
Vacant Land       1.086689
Name: Sales Ratio, dtype: float64


In [14]:
# set the outlier removal threshold
z_threshold = 3

# group the dataframe by 'Property Type'
grouped = df.groupby('Property Type')

# define a function to remove outliers based on the Z-Score method
def remove_outliers_zscore(data):
    z_scores = stats.zscore(data)
    return data[abs(z_scores) < z_threshold]

# calculate the average sales ratio for each property type
result = grouped['Sales Ratio'].apply(remove_outliers_zscore).groupby('Property Type').mean()

# print the result
print(result)


Property Type
Apartments        0.523314
Commercial        2.565129
Condo             0.775733
Four Family       1.220069
Industrial        1.004313
Public Utility    0.955225
Residential       0.603756
Single Family     1.029146
Three Family      1.205067
Two Family        1.072740
Vacant Land       0.836763
Name: Sales Ratio, dtype: float64


In [15]:
def replace_property_type(sales_ratio):
    if sales_ratio < 0.5233140930291666:
        return "Apartments"
    elif 0.60375565641728 > sales_ratio >= 0.5233140930291666:
        return "Residential"
    elif 0.7757329201923256 > sales_ratio >= 0.60375565641728:
        return "Condo"
    elif 0.8367634436023521 > sales_ratio >= 0.7757329201923256:
        return "Vacant Land"
    elif 0.9552250986 > sales_ratio >= 0.8367634436023521:
        return "Public Utility"
    elif 1.0043125217743363 > sales_ratio >= 0.9552250986:
        return "Industrial"
    elif 1.0291462009565886 > sales_ratio >= 1.0043125217743363:
        return "Single Family"
    elif 1.072740218830206 > sales_ratio >= 1.0291462009565886:
        return "Three Family"
    elif 1.205066695693422 > sales_ratio >= 1.072740218830206:
        return "Four Family"
    elif 1.2200687653968179 > sales_ratio >= 1.205066695693422:
        return "Other"
    else:
        return "Commercial"

df["Property Type"] = df["Property Type"].fillna(df["Sales Ratio"].apply(replace_property_type))


In [16]:
df.head(5)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020348,2020,09/13/2021,Ansonia,WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
1,20002,2020,10/02/2020,Ashford,TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,NaN,NaN,NaN,NaN
2,200212,2020,03/09/2021,Avon,CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,NaN,NaN,NaN,NaN
3,200243,2020,04/13/2021,Avon,NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,NaN,NaN,NaN,NaN
4,200377,2020,07/02/2021,Avon,FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,NaN,NaN,NaN,NaN


In [17]:
df['Date Recorded'] = pd.to_datetime(df['Date Recorded'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997161 entries, 0 to 997212
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Serial Number     997161 non-null  int64         
 1   List Year         997161 non-null  int64         
 2   Date Recorded     997161 non-null  datetime64[ns]
 3   Town              997161 non-null  object        
 4   Address           997161 non-null  object        
 5   Assessed Value    997161 non-null  float64       
 6   Sale Amount       997161 non-null  float64       
 7   Sales Ratio       997161 non-null  float64       
 8   Property Type     997161 non-null  object        
 9   Residential Type  608899 non-null  object        
 10  Non Use Code      289632 non-null  object        
 11  Assessor Remarks  149863 non-null  object        
 12  OPM remarks       9934 non-null    object        
 13  Location          197692 non-null  object        
dtypes: d

In [19]:
df['Date_Recorded_year'] = df['Date Recorded'].dt.year
df['Date_Recorded_month'] = df['Date Recorded'].dt.month
df['Date_Recorded_day'] = df['Date Recorded'].dt.day
df = df.drop(['Serial Number', 'Date Recorded', 'Non Use Code', 'Assessor Remarks', 'OPM remarks', 'Location'], axis=1)

In [20]:
df.head()

,List Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Date_Recorded_year,Date_Recorded_month,Date_Recorded_day
0,2020,Ansonia,WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,2021,9,13
1,2020,Ashford,TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,2020,10,2
2,2020,Avon,CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,2021,3,9
3,2020,Avon,NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,2021,4,13
4,2020,Avon,FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,2021,7,2


ML

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# encode the 'Fruit' column
df['Property_Type'] = le.fit_transform(df['Property Type'])
df['Town_Encoded'] = le.fit_transform(df['Town'])
df['Residential_Type'] = le.fit_transform(df['Residential Type'])
df['Address_encoded'] = le.fit_transform(df['Address'])

In [22]:
df.head()

,List Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Date_Recorded_year,Date_Recorded_month,Date_Recorded_day,Property_Type,Town_Encoded,Residential_Type,Address_encoded
0,2020,Ansonia,WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,2021,9,13,1,1,5,96684
1,2020,Ashford,TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,2020,10,2,7,2,2,94242
2,2020,Avon,CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,2021,3,9,7,3,0,20322
3,2020,Avon,NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,2021,4,13,7,3,2,65489
4,2020,Avon,FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,2021,7,2,7,3,2,32960


In [23]:
fd = df[['List Year','Date_Recorded_day','Date_Recorded_month', 'Date_Recorded_year', 'Town_Encoded', 'Address_encoded','Assessed Value', 'Sale Amount','Sales Ratio', 'Property_Type', 'Residential_Type']]

In [ ]:
fd = df[['List Year','Date_Recorded_day','Date_Recorded_month', 'Date_Recorded_year', 'Town_Encoded', 'Address_encoded','Assessed Value', 'Sale Amount','Sales Ratio', 'Property_Type', 'Residential_Type']]

In [24]:
fd.head()

,List Year,Date_Recorded_day,Date_Recorded_month,Date_Recorded_year,Town_Encoded,Address_encoded,Assessed Value,Sale Amount,Sales Ratio,Property_Type,Residential_Type
0,2020,13,9,2021,1,96684,150500.0,325000.0,0.4630,1,5
1,2020,2,10,2020,2,94242,253000.0,430000.0,0.5883,7,2
2,2020,9,3,2021,3,20322,130400.0,179900.0,0.7248,7,0
3,2020,13,4,2021,3,65489,619290.0,890000.0,0.6958,7,2
4,2020,2,7,2021,3,32960,862330.0,1447500.0,0.5957,7,2


In [ ]:
fd.info()

In [ ]:
X = fd.drop(['Sale Amount'], axis=1)
y = fd['Sale Amount']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Compute mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.2f}")

# Compute R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.2f}")


In [ ]:
# Split data into train and test sets
X = fd.drop(['Sale Amount'], axis=1)
y = fd['Sale Amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train, y_train)

# Make predictions on test set
y_pred = gbm.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'R^2: {r2:.2f}')


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create the model and fit the training data
tree_reg = DecisionTreeRegressor(max_depth=18)
tree_reg.fit(X_train, y_train)
# Predict on test data
y_pred = tree_reg.predict(X_test)

 # Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'R^2: {r2:.2f}')


In [ ]:
from sklearn.tree import export_graphviz

# Export decision tree to DOT file
export_graphviz(tree_reg, out_file='tree.dot', feature_names=X_train.columns)

In [ ]:
from joblib import dump

# Save the model to a file
dump(tree_reg, 'tree_reg.joblib')


In [ ]:
from sklearn import tree
tree.plot_tree(tree_reg);